# EDA: Registre d'explotacions ramaderes

In [1]:
# Load packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Informació del *dataset*

El *dataset* escollit és el **Registre d'explotacions ramaderes de Catalunya**, que es pot trobar a [Dades Obertes de Catalunya](https://analisi.transparenciacatalunya.cat/Medi-Rural-Pesca/Registre-d-explotacions-ramaderes/7bpt-5azk). Aquest *dataset* conté informació sobre les explotacions ramaderes de Catalunya, amb dades com la seva ubicació, el tipus d'explotació, el nombre d'animals que hi ha, etc. Dintre de cadascuna de les seccions, les explotacions es classifiquen en diferents subseccions en funció de la finalitat productiva.

Les dades les proporciona el **Departament d'Acció Climàtica, Alimentació i Agenda Rural**. Aquest s'encarrega d'actualitzar les dades anualment. L'última actualització és del 8 de novembre del 2023, per tant, les dades són ben actuals.

El joc de dades conté més de 26.000 registres i 49 variables. Cada fila és una explotació ramadera i les columnes que conté mostren informació identificativa de les explotacions i característiques de les mateixes. Les variables són de tipus numèric, de text i de data.

Totes les dades que es mostren són públiques. De totes maneres, hi ha informació sobre empreses privades i particulars, per tant, s'ha de tenir cura amb la privacitat de les dades.

A continuació carreguem les dades, en visualitzem l'estructura i mostrem les primeres files:

In [3]:
# Load dataset
df = pd.read_csv('../data/Registre_d_explotacions_ramaderes.csv')

In [8]:
# Convert postal code to object
df['Codi postal explotació'] = df['Codi postal explotació'].astype('object')

# Convert date columns to datetime
df['Data canvi estat explotació'] = pd.to_datetime(df['Data canvi estat explotació'], format='%d/%m/%Y')
df['Data canvi estat subexplotació'] = pd.to_datetime(df['Data canvi estat subexplotació'], format='%d/%m/%Y')
df['Data actualització capacitat'] = pd.to_datetime(df['Data actualització capacitat'], format='%d/%m/%Y')

# See the structure of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25998 entries, 0 to 25997
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   Marca oficial                   25998 non-null  object        
 1   Codi REGA                       25998 non-null  object        
 2   Estat explotació                25998 non-null  object        
 3   Data canvi estat explotació     25998 non-null  datetime64[ns]
 4   Nom explotació                  25998 non-null  object        
 5   Adreça explotació               25998 non-null  object        
 6   Codi postal explotació          25998 non-null  object        
 7   Servei territorial explotació   25998 non-null  object        
 8   Província explotació            25998 non-null  object        
 9   Comarca explotació              25998 non-null  object        
 10  Municipi explotació             25998 non-null  object        
 11  CO

In [9]:
# See the first rows of the dataset
df.head()

,Marca oficial,Codi REGA,Estat explotació,Data canvi estat explotació,Nom explotació,Adreça explotació,Codi postal explotació,Servei territorial explotació,Província explotació,Comarca explotació,...,Capacitat transició,Capacitat estants,Capacitat transhumants,Capacitat AN>6M no repr,Capacitat ous,Cap núm. total animals,Total Cap ponedores,Total URM,Total Nitrogen,Data actualització capacitat
0,0010AA,ES250010030750,Activa,1997-01-21,CASA JULIU - JORDI GASA,MASIA JULIU,25651,Alt Pirineu,Lleida,Pallars Jussà,...,0,0,0,0,0,0,0,56,7515,2021-10-04
1,0010AA,ES250010030750,Activa,1997-01-21,CASA JULIU - JORDI GASA,MASIA JULIU,25651,Alt Pirineu,Lleida,Pallars Jussà,...,0,0,0,0,0,0,0,29088,17574,2023-07-21
2,0010AA,ES250010030750,Activa,1997-01-21,CASA JULIU - JORDI GASA,MASIA JULIU,25651,Alt Pirineu,Lleida,Pallars Jussà,...,0,0,0,0,0,0,0,4,2552,2014-11-21
3,0010AA,ES250010030750,Activa,1997-01-21,CASA JULIU - JORDI GASA,MASIA JULIU,25651,Alt Pirineu,Lleida,Pallars Jussà,...,0,0,0,4,0,0,0,4,2552,2014-11-21
4,0010AB,ES250010031100,Activa,1997-04-21,CASA GONELLA,BOIXOLS,25651,Alt Pirineu,Lleida,Pallars Jussà,...,475,0,0,0,0,0,0,19702,1187525,2021-10-04


## Preparació de les dades

En aquest apartat examinarem el dataset per tal d'esbrinar si el conjunt de dades conté valors perduts, valors extrems, etc. Estudiarem si cal homogeneïtzar les dades categòriques i si caldrà extreure característiques.